In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [3]:
ic50 = pd.read_csv(
    "../data/drug/DrugResponse_PANCANCER_GDSC1_GDSC2_IC_20191119.csv")
ic50 = ic50.sort_values(by=['Dataset version'])
ic50 = ic50.drop_duplicates(
    ['Drug Id', 'Cell line name'], keep='last').sort_values(
        by=['Drug Id', 'Cell line name']).reset_index(drop=True)

In [14]:
ic50['IC50_norm'] = 0

In [15]:
for drug in ic50['Drug Id'].unique():
    scaler = MinMaxScaler()
    ic50.loc[ic50['Drug Id'] == drug, 'IC50_norm'] = scaler.fit_transform(
        ic50[ic50['Drug Id'] == drug]['IC50'].values.reshape(-1, 1))

In [17]:
tmp = ic50.groupby(['Drug Id'])['IC50_norm'].agg(np.median).reset_index()
ic50_median = dict(zip(tmp['Drug Id'].values, tmp['IC50_norm'].values))

In [18]:
ic50['rel_sensitive'] = ic50.apply(
    lambda row: row['IC50_norm'] < ic50_median[row['Drug Id']], axis=1)
ic50['rel_sensitive'] = ic50['rel_sensitive'].astype(int)

In [19]:
ic50['rel_sensitive'].describe()

count    363120.000000
mean          0.499716
std           0.500001
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max           1.000000
Name: rel_sensitive, dtype: float64

In [23]:
ic50.to_csv("../data/drug/ic50_processed_median.csv", index=False)

In [22]:
ic50['IC50_norm'].describe()

count    363120.000000
mean          0.544308
std           0.194840
min           0.000000
25%           0.405692
50%           0.553786
75%           0.693729
max           1.000000
Name: IC50_norm, dtype: float64